In [ ]:
import os
import sys

from shared import load_csv_from_dir, post_to_kafka_bridge

from dotenv import load_dotenv
load_dotenv()


In [ ]:
START = '2020-04-01'
END = '2020-04-10'


In [ ]:
def upload_fraud(bridge, topic='tx-fraud-sim', start='2020-04-01', end='2020-04-01', loc='./data/synthetic/fraud/', batch_size=100):

    KAFKA_ENDPOINT = f"{bridge}/topics/{topic}"

    # read the raw transaction data
    transactions_df = load_csv_from_dir(loc, start, end)
    NUM_TX = len(transactions_df)

    batch = []

    for index, row in transactions_df.iterrows():
        batch.append(row)

        if len(batch) % batch_size == 0:
            post_to_kafka_bridge(KAFKA_ENDPOINT, batch)

            batch = []
            print(f" --> uploaded {index+1}/{NUM_TX}")

    if len(batch) > 0:
        post_to_kafka_bridge(KAFKA_ENDPOINT, batch)
        print(f" --> uploaded remaining {len(batch)}/{NUM_TX}")


In [ ]:
bridge_endpoint = os.getenv('kafka_bridge_endpoint')
if bridge_endpoint == None:
    print("Missing ENV['kafka_bridge_endpoint']")
    sys.exit()
else:
    bridge_endpoint = bridge_endpoint.lstrip('"').rstrip('"')
    print(f"Using endpoint '{bridge_endpoint}'")


In [ ]:
print(f" --> Uploading fraud data from {START} to {END}")

upload_fraud(bridge_endpoint, start=START, end=END)

print(" --> DONE.")
